# fMRI Smoothing with NILearn in Parallel 

  By Kyle Baacke
  3/13/2022

## Description:
  You may find yourself wanting to run a process in parallel (i.e. multiple processes running simultaneously) within a python script. In this example, we use a parallel for loop in python to smooth fMRI images using nilearn. The process can be divided into 3 main steps: 1) setting up string patterns to read and write files, 2) defining a function to run in parallel, and 3) calling the function in parallel. This snippet will go through each process with an example which callas a python script using variables in nested for loops.

  In this example, we ierate through each subject available from the HCP_1200 dataset, read in each of their task-fMRI sessions, smooth those 4 dimensional images using nilearn, and save the smoothed images.
    
## Notes and Qualifiers:
  This code has been written and tested with Python 3.8 and 3.9 on Ubuntu, Windows 10, and OpenSUSE. 

## Helpful Links
  https://nilearn.github.io/modules/generated/nilearn.image.smooth_img.html
  https://joblib.readthedocs.io/en/latest/parallel.html
  
## 0) Setup

### Imports

In [2]:
from nilearn import plotting, image
import datetime as dt
import os
from joblib import Parallel, delayed
sep = os.path.sep
# I use sep instead of manually typing the path to make changing between OSs easier

## 1) Set up String Patterns

In [5]:
# Path to dataset
DATA_PATH = f'S:{sep}HCP{sep}HCP_1200'
# Pattern of path to functional scan
NIFTI_PATH = '{outdir}{sep}{SUBJECT}{sep}MNINonLinear{sep}Results{sep}{TIME}_{RUN}{sep}{TIME}_{RUN}.nii.gz'
# Pattern of ourpur path and filename
OUTPUT_PATH = '{outdir}{sep}{SUBJECT}{sep}MNINonLinear{sep}Results{sep}{TIME}_{RUN}{sep}{TIME}_{RUN}_nilearn-smoothed.nii.gz'
# List of subject folders in DATA_PATH
HCP_subject_list = os.listdir(DATA_PATH)
# List session names to iterate through
# In this case, we only want to run on the Worrking Memory session (WM), so the other sessions in the list are commented out
HCP_Session_list = [
    # "tfMRI_MOTOR",
    "tfMRI_WM"#,
    # "tfMRI_EMOTION",
    # "tfMRI_GAMBLING",
    # "tfMRI_LANGUAGE",
    # "tfMRI_RELATIONAL",
    # "tfMRI_SOCIAL"
  ]
# List run identifiers
HCP_Run_list = ['RL','LR']

## 2) Defining function
  The nilearn smooth_img function does most of the heavy lifting here, but we want to make things even easier and bundle in a running measure of the storage space used by this operation. In this case, we will only run in parallel on the subject level. However, it is just as manageable to parallelize on the level of the session or the session and the run. 

In [7]:
def smooth_subject(SUBJECT):
  # Create and empty list to store the file sizes as we go
  size_list = []
  # Iterate through the lists of strings we created in step 1
  for TIME in HCP_Session_list:
    for RUN in HCP_Run_list:
      # Format the patterns we created before to target specific runs
      input_path = NIFTI_PATH.format(
        outdir = DATA_PATH,
        SUBJECT = SUBJECT,
        TIME = TIME,
        RUN = RUN,
        sep = sep
      )
      output_path = OUTPUT_PATH.format(
        outdir = DATA_PATH,
        SUBJECT = SUBJECT,
        TIME = TIME,
        RUN = RUN,
        sep = sep
      )
      try:
        smooth_img = image.smooth_img(input_path, 6)
        smooth_img.to_filename(output_path)
        size_list.append(os.path.getsize(output_path))
      except:
        print(f'File not found: {output_path}')
  return sum(size_list)

## 3) Call Function in Parallel

In [8]:
# Make note of start time
start_time = dt.datetime.now()
# Save the output of the function calls called in parallel to a list
# ^^^^^^^   Call the Parallel fuunction with n_jobs={the number of threads you want to use}
#    |      ^^^^^^^^^^^^^^^^^  Call the function 'delayed' until the previous 6 jobs have completed
#    |              |          ^^^^^^^ the function you want to run
#    |              |             |    ^^^^^^^^^^^^^^  the variable you want to pass to the function
#    |              |             |          |         ^^^^^^^  Assigning the variable using a for loop
#    |              |             |          |            |     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
size_list = Parallel(n_jobs=6)(delayed(smooth_subject)(SUBJECT) for SUBJECT in HCP_subject_list[:6])
# Note the end time
end_time = dt.datetime.now()
# Print the runtime and filesize used to the console
print('Done. ', end_time-start_time, round(float(sum(size_list))/1000000000.0, 4), ' GB')

Done.  0:04:01.568659 7.6383  GB
